In [1]:
# The code was removed by Watson Studio for sharing.

# Introduction/Business Problem
### A description of the problem and a discussion of the background.
Taking on the role of a restauranteur; I will use my new data science knowledge to help in the location planning phase for a new venture in a wealthy, fast growing city inside the United States.  

# Data Plan

### A description of the data and how it will be used to solve the problem.

To solve this challenge, the following preliminary plan is in place.
1. Determine the proper cities/areas which are both wealthy and fast growing.
2. Determine the most common resturaunt types across the chosen area.
3. Use clustering to the identify most consentrated returaunt hubs. (Areas of a fixed size where resturaunts are present in higher than average density.)
4. Create a score for each hub, determined based on the resuraunt type with both the highest instance across the entire target area, but the lowest presence within the hub.
5. Present the top 10 from this scoring as a list of opportunities.
6. Perform some a manual review of the results and select 3 to present as the top 3 suggestions for a new buisness.


In support of that plan, the following data sources will be used.

Census data:
* The expected wealth of populations in a city
* The recent growth rate of the cities being reviewed

Foursquare:
* Identify the different types of restaurants across the target area

Other:
* Geo-spacial location data from cocl.us will be used for visualizations

All of the data will be compiled and appropriate visualizations will be produced and reviewed.  A final determination will be made by hand.



# 1. Determine the proper cities/areas which are both wealthy and fast growing.




The list below was created by combining the data from the US Census Bureau with population data and sorted based on Population, growth rate, and income (with priorities in that order)


#### Massachusetts: Boston-Cambridge-Newton
- Approx Population: ***4,470k***
- 2010-2018 pop. growth: **6.8%**
- Median household income: **\\$85,691**

#### Connecticut: Bridgeport-Stamford-Norwalk
- Approx Population: **950k**
- 2010-2018 pop. growth: **2.6%**
- Median household income: **\\$91,198**

#### New Jersey: Trenton
- Approx Population: **85k**
- 2010-2018 pop. growth: **0.5%**
- Median household income: **\\$79,173**

#### New Hampshire: Manchester-Nashua
- Approx Population: **88k**
- 2010-2018 pop. growth: **3.5%**
- Median household income: **\\$78,769**

#### Hawaii: Kahului-Wailuku-Lahaina
- Approx Population: **12k**
- 2010-2018 pop. growth: **7.9%**
- Median household income: **\\$80,183**

---

**Boston-Cambridge-Newton, MA-NH Metro Area** area was selected due to it's, comparitively, massive population and high growth rate.  
Bridgeport-Stamford-Norwalk, CT, with it's high population and higher median income would be a reasonable second choice.  But it's 2.6\% growth rate is worrisome.


(Source: https://www.usatoday.com/story/money/2019/04/22/the-fastest-growing-city-in-each-state/39362311/)

### Get relevant zip codes.

To get the zip codes into a list that can be used for further queries; the data from https://www.bestplaces.net/find/zip.aspx?msa=14460&st=MA was placed into a text file and processed below.

In [2]:
#hard coding center of Greater Boston for later use
greaterBoston_lat=42.35817
greaterBoston_lng=-71.06369

In [3]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

import numpy as np

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
kclusters = 5

In [4]:
# The code was removed by Watson Studio for sharing.

,town,near
0,Abington,"""Abington, MA"""
1,Acton,"""Acton, MA"""
2,Amesbury,"""Amesbury, MA"""
3,Andover,"""Andover, MA"""
4,Arlington,"""Arlington, MA"""


### Initialize the subroutines and other parts needed for the remainder of this project.

In [5]:
# The code was removed by Watson Studio for sharing.

In [6]:
!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv

--2020-05-23 01:16:34--  https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Resolving cocl.us (cocl.us)... 158.85.108.86, 158.85.108.83, 169.48.113.194
Connecting to cocl.us (cocl.us)|158.85.108.86|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-05-23 01:16:36--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.29.197
Connecting to ibm.box.com (ibm.box.com)|107.152.29.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-05-23 01:16:36--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr8

In [7]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment for the first run of this notebook as needed.
import folium # map rendering library

from pandas.io.json import json_normalize # tranforms JSON files into a pandas dataframes

import requests

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                       

In [8]:
allresults = []

# reuse the get_category_type and getNearbyVenues functions from the lab.

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

def getNearbyVenuesNear(names, near, radius=5000, limit=500):
    venues_list=[]
    for name, thisnear in zip(names, near):
        #print(name) #Rem'd since it is uneeded unless testing.

        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&radius={}&limit={}'.format(
            CLIENTID, 
            CLIENTSECRET, 
            CLIENTVERSION, 
            thisnear, 
            radius, 
            limit)
        
        print(url)
        
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        allresults.append(results)
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            v['venue']['name'],    
            v['venue']['location']['lng'], 
            v['venue']['location']['lat'], 
            v['venue']['categories'][0]['name']) for v in results])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = [
            'neighbourhood',
            'name',    
            'lng', 
            'lat', 
            'catname']
        
    #Return the completed 
    return(nearby_venues)

# 2. Determine the most common resturaunt types across the chosen area.

### Build a profile of the existing restauraunts

Using the zip codes that were just read, foursquare is querried to find the most visited resturaunts in the area.

In [9]:
### ½ DEV CODE ###
###Un-REM when running for the first time. ###
### ½ DEV CODE ###

#Pull the data and store a copy of the resultant dataframe so we can manipulate it without having to re-query the api during dev.
df_bcn_venues=pd.DataFrame(getNearbyVenuesNear(bcn_near['town'],bcn_near['near']))
df_bcn_venues_bkp=pd.DataFrame(df_bcn_venues)

df_bcn_venues

https://api.foursquare.com/v2/venues/explore?&client_id=K35LHC34SX1ZW2M4JOVPEFYOMBVVJOOZMCOTNYNQPBJN3OZ4&client_secret=AUQ4CADYTEURBZ3WC3KAM24WOG55HBWZHPRTLWS2AUXUENJG&v=20170511&near="Abington, MA"&radius=5000&limit=500
https://api.foursquare.com/v2/venues/explore?&client_id=K35LHC34SX1ZW2M4JOVPEFYOMBVVJOOZMCOTNYNQPBJN3OZ4&client_secret=AUQ4CADYTEURBZ3WC3KAM24WOG55HBWZHPRTLWS2AUXUENJG&v=20170511&near="Acton, MA"&radius=5000&limit=500
https://api.foursquare.com/v2/venues/explore?&client_id=K35LHC34SX1ZW2M4JOVPEFYOMBVVJOOZMCOTNYNQPBJN3OZ4&client_secret=AUQ4CADYTEURBZ3WC3KAM24WOG55HBWZHPRTLWS2AUXUENJG&v=20170511&near="Amesbury, MA"&radius=5000&limit=500
https://api.foursquare.com/v2/venues/explore?&client_id=K35LHC34SX1ZW2M4JOVPEFYOMBVVJOOZMCOTNYNQPBJN3OZ4&client_secret=AUQ4CADYTEURBZ3WC3KAM24WOG55HBWZHPRTLWS2AUXUENJG&v=20170511&near="Andover, MA"&radius=5000&limit=500
https://api.foursquare.com/v2/venues/explore?&client_id=K35LHC34SX1ZW2M4JOVPEFYOMBVVJOOZMCOTNYNQPBJN3OZ4&client_secret=A

,neighbourhood,name,lng,lat,catname
0,Abington,10th District Brewery,-70.945963,42.105673,Brewery
1,Abington,Spencer's Pizza,-70.939848,42.106542,Pizza Place
2,Abington,Dairy Queen,-70.931984,42.109693,Ice Cream Shop
3,Abington,Brother's Roast Beef and Pizza,-70.945855,42.106271,American Restaurant
4,Abington,Starbucks,-70.930550,42.110790,Coffee Shop
5,Abington,Wendy’s,-70.930148,42.110798,Fast Food Restaurant
6,Abington,Mia Regazza,-70.950182,42.100615,Italian Restaurant
7,Abington,Island Grove Park,-70.937176,42.112455,Dog Run
8,Abington,OLD TOWN CAFE,-70.941587,42.128035,American Restaurant
9,Abington,Midway Automotive,-70.959214,42.100908,Automotive Shop


In [10]:
### ½ DEV CODE ###
#reset the dataframe to clear any bad manipulations during dev.

df_bcn_venues=pd.DataFrame(df_bcn_venues_bkp)
df_bcn_venues.head(3)

,neighbourhood,name,lng,lat,catname
0,Abington,10th District Brewery,-70.945963,42.105673,Brewery
1,Abington,Spencer's Pizza,-70.939848,42.106542,Pizza Place
2,Abington,Dairy Queen,-70.931984,42.109693,Ice Cream Shop


### Perform a manual review of the venue types ('catname') and determine which are in scope for this project and how to group them into more helpful buckets for analysis.  Then, merge that information with the retreived dataframe for further analysis.

In [11]:
print('There are {} unique categories.'.format(len(df_bcn_venues['catname'].unique())))
print(df_bcn_venues['catname'].unique())

There are 374 unique categories.
['Brewery' 'Pizza Place' 'Ice Cream Shop' 'American Restaurant'
 'Coffee Shop' 'Fast Food Restaurant' 'Italian Restaurant' 'Dog Run'
 'Automotive Shop' 'Liquor Store' 'State / Provincial Park' 'Bar'
 'Sandwich Place' 'Breakfast Spot' 'Café' 'Convenience Store' 'Donut Shop'
 'Pharmacy' 'Hardware Store' 'Shipping Store' 'Big Box Store'
 'Historic Site' 'Pub' 'Chinese Restaurant' 'Bowling Alley'
 'Farmers Market' 'Brazilian Restaurant' 'Sushi Restaurant' 'Supermarket'
 'Restaurant' 'Hobby Shop' 'Theme Park' 'Gift Shop' 'Salon / Barbershop'
 'Rental Car Location' 'Discount Store' 'Video Game Store'
 'Mexican Restaurant' 'Gym' 'Fried Chicken Joint' 'Asian Restaurant'
 'Arcade' 'Grocery Store' 'Bookstore' 'Gym / Fitness Center' 'Laundromat'
 'Trail' 'Museum' 'Food & Drink Shop' 'Martial Arts Dojo' 'Pet Store'
 'Bagel Shop' 'Bakery' 'Health Food Store' 'New American Restaurant'
 'Kids Store' 'Department Store' 'Hotel' 'Burrito Place'
 'Seafood Restaurant' 'Ska

In [12]:
# scopelist was manually created based on the goals of this project.
scopelist=[['American Restaurant','Restaurant'],['Arepa Restaurant','Restaurant'],['Argentinian Restaurant','Restaurant'],['Asian Restaurant','Restaurant'],['Bagel Shop','Food Shop'],['Bakery','Food Shop'],['BBQ Joint','Restaurant'],['Brewery','Barcentric'],['Belgian Restaurant','Restaurant'],['Bistro','Restaurant'],['Bar','Barcentric'],['Pub','Barcentric'],['Brazilian Restaurant','Restaurant'],['Breakfast Spot','Restaurant'],['Bubble Tea Shop','Food Shop'],['Buffet','Restaurant'],['Burger Joint','Grab and Go'],['Café','Food Shop'],['Cafeteria','Other'],['Cajun / Creole Restaurant','Restaurant'],['Cambodian Restaurant','Restaurant'],['Candy Store','Food Shop'],['Caribbean Restaurant','Restaurant'],['Cheese Shop','Food Shop'],['Chinese Restaurant','Restaurant'],['Chocolate Shop','Food Shop'],['Coffee Shop','Food Shop'],['Comfort Food Restaurant','Restaurant'],['Creperie','Food Shop'],['Cuban Restaurant','Restaurant'],['Cupcake Shop','Food Shop'],['Deli / Bodega','Food Shop'],['Dessert Shop','Food Shop'],['Diner','Restaurant'],['Dive Bar','Barcentric'],['Donut Shop','Food Shop'],['Dumpling Restaurant','Restaurant'],['Eastern European Restaurant','Restaurant'],['Ethiopian Restaurant','Restaurant'],['Falafel Restaurant','Restaurant'],['Fast Food Restaurant','Restaurant'],['Food','Other'],['Food & Drink Shop','Food Shop'],['Food Court','Other'],['Food Truck','Grab and Go'],['French Restaurant','Restaurant'],['Fried Chicken Joint','Grab and Go'],['Wine Shop','Barcentric'],['Frozen Yogurt Shop','Food Shop'],['Gaming Cafe','Other'],['Gastropub','Restaurant'],['Greek Restaurant','Restaurant'],['Hot Dog Joint','Grab and Go'],['Ice Cream Shop','Food Shop'],['Lounge','Barcentric'],['Cocktail Bar','Barcentric'],['Indian Chinese Restaurant','Restaurant'],['Speakeasy','Barcentric'],['Indian Restaurant','Restaurant'],['Italian Restaurant','Restaurant'],['Japanese Restaurant','Restaurant'],['Juice Bar','Food Shop'],['Korean Restaurant','Restaurant'],['Irish Pub','Barcentric'],['Kosher Restaurant','Restaurant'],['Latin American Restaurant','Restaurant'],['Malay Restaurant','Restaurant'],['Mediterranean Restaurant','Restaurant'],['Mexican Restaurant','Restaurant'],['Meze Restaurant','Restaurant'],['Middle Eastern Restaurant','Restaurant'],['New American Restaurant','Restaurant'],['Noodle House','Restaurant'],['Peking Duck Restaurant','Restaurant'],['Persian Restaurant','Restaurant'],['Pizza Place','Grab and Go'],['Poke Place','Grab and Go'],['Portuguese Restaurant','Restaurant'],['Ramen Restaurant','Restaurant'],['Restaurant','Restaurant'],['Salad Place','Grab and Go'],['Sandwich Place','Grab and Go'],['Seafood Restaurant','Restaurant'],['Snack Place','Grab and Go'],['Soup Place','Grab and Go'],['Sake Bar','Barcentric'],['South Indian Restaurant','Restaurant'],['Southern / Soul Food Restaurant','Restaurant'],['Karaoke Bar','Barcentric'],['Nightclub','Barcentric'],['Spanish Restaurant','Restaurant'],['Steakhouse','Restaurant'],['Sushi Restaurant','Restaurant'],['Beer Garden','Barcentric'],['Wine Bar','Barcentric'],['Szechuan Restaurant','Restaurant'],['Taco Place','Restaurant'],['Taiwanese Restaurant','Restaurant'],['Tapas Restaurant','Restaurant'],['Tea Room','Restaurant'],['Tex-Mex Restaurant','Restaurant'],['Thai Restaurant','Restaurant'],['Turkish Restaurant','Restaurant'],['Jazz Club','Barcentric'],['Vegetarian / Vegan Restaurant','Restaurant'],['Venezuelan Restaurant','Restaurant'],['Vietnamese Restaurant','Restaurant'],['Wings Joint','Restaurant'],['Convenience Store','OUT OF SCOPE'],['Liquor Store','OUT OF SCOPE'],['Trail','OUT OF SCOPE'],['Grocery Store','OUT OF SCOPE'],['Kids Store','OUT OF SCOPE'],['Womens Store','OUT OF SCOPE'],['Thrift / Vintage Store','OUT OF SCOPE'],['Soccer Field','OUT OF SCOPE'],['Massage Studio','OUT OF SCOPE'],['Beer Store','OUT OF SCOPE'],['Toy / Game Store','OUT OF SCOPE'],['Indie Movie Theater','OUT OF SCOPE'],['Yoga Studio','OUT OF SCOPE'],['Home Service','OUT OF SCOPE'],['Park','OUT OF SCOPE'],['Sporting Goods Shop','OUT OF SCOPE'],['Pharmacy','OUT OF SCOPE'],['Shipping Store','OUT OF SCOPE'],['Bank','OUT OF SCOPE'],['Clothing Store','OUT OF SCOPE'],['Arts & Crafts Store','OUT OF SCOPE'],['Bookstore','OUT OF SCOPE'],['Farmers Market','OUT OF SCOPE'],['Theater','OUT OF SCOPE'],['Performing Arts Venue','OUT OF SCOPE'],['Spa','OUT OF SCOPE'],['Concert Hall','OUT OF SCOPE'],['Market','OUT OF SCOPE'],['Historic Site','OUT OF SCOPE'],['Campground','OUT OF SCOPE'],['Miscellaneous Shop','OUT OF SCOPE'],['Gym','OUT OF SCOPE'],['Business Service','OUT OF SCOPE'],['Playground','OUT OF SCOPE'],['Moving Target','OUT OF SCOPE'],['Department Store','OUT OF SCOPE'],['Salon / Barbershop','OUT OF SCOPE'],['Tailor Shop','OUT OF SCOPE'],['Athletics & Sports','OUT OF SCOPE'],['ATM','OUT OF SCOPE'],['Pet Store','OUT OF SCOPE'],['Hobby Shop','OUT OF SCOPE'],['Plaza','OUT OF SCOPE'],['Train','OUT OF SCOPE'],['Event Service','OUT OF SCOPE'],['Bus Stop','OUT OF SCOPE'],['Automotive Shop','OUT OF SCOPE'],['Furniture / Home Store','OUT OF SCOPE'],['Hardware Store','OUT OF SCOPE'],['Gas Station','OUT OF SCOPE'],['Baseball Field','OUT OF SCOPE'],['Library','OUT OF SCOPE'],['Circus','OUT OF SCOPE'],['Hotel','OUT OF SCOPE'],['Boxing Gym','OUT OF SCOPE'],['Gym / Fitness Center','OUT OF SCOPE'],['History Museum','OUT OF SCOPE'],['Tourist Information Center','OUT OF SCOPE'],['Outdoor Sculpture','OUT OF SCOPE'],['Forest','OUT OF SCOPE'],['Music Venue','OUT OF SCOPE'],['Cosmetics Shop','OUT OF SCOPE'],['Dry Cleaner','OUT OF SCOPE'],['Smoke Shop','OUT OF SCOPE'],['Marijuana Dispensary','OUT OF SCOPE'],['Storage Facility','OUT OF SCOPE'],['Flower Shop','OUT OF SCOPE'],['Gourmet Shop','OUT OF SCOPE'],['Sports Bar','OUT OF SCOPE'],['Knitting Store','OUT OF SCOPE'],['Bed & Breakfast','OUT OF SCOPE'],['Train Station','OUT OF SCOPE'],['Farm','OUT OF SCOPE'],['Art Gallery','OUT OF SCOPE'],['Video Store','OUT OF SCOPE'],['Shop & Service','OUT OF SCOPE'],['Construction & Landscaping','OUT OF SCOPE'],['Bike Trail','OUT OF SCOPE'],['Rest Area','OUT OF SCOPE'],['Laundromat','OUT OF SCOPE'],['Discount Store','OUT OF SCOPE'],['Indie Theater','OUT OF SCOPE'],['Shopping Mall','OUT OF SCOPE'],['Boutique','OUT OF SCOPE'],['Basketball Court','OUT OF SCOPE'],['Dance Studio','OUT OF SCOPE'],['Electronics Store','OUT OF SCOPE'],['Lawyer','OUT OF SCOPE'],['Intersection','OUT OF SCOPE'],['Paper / Office Supplies Store','OUT OF SCOPE'],['Bike Shop','OUT OF SCOPE'],['Warehouse Store','OUT OF SCOPE'],['Health & Beauty Service','OUT OF SCOPE'],['Mobile Phone Shop','OUT OF SCOPE'],['Supplement Shop','OUT OF SCOPE'],['Gift Shop','OUT OF SCOPE'],['Outdoors & Recreation','OUT OF SCOPE'],['High School','OUT OF SCOPE'],['Mattress Store','OUT OF SCOPE'],['Video Game Store','OUT OF SCOPE'],['Track','OUT OF SCOPE'],['Mens Store','OUT OF SCOPE'],['Shoe Store','OUT OF SCOPE'],['Museum','OUT OF SCOPE'],['Golf Course','OUT OF SCOPE'],['Lake','OUT OF SCOPE'],['Sports Club','OUT OF SCOPE'],['Credit Union','OUT OF SCOPE'],['Doctors Office','OUT OF SCOPE'],['Hotel Pool','OUT OF SCOPE'],['Garden Center','OUT OF SCOPE'],['Harbor / Marina','OUT OF SCOPE'],['Multiplex','OUT OF SCOPE'],['Fabric Shop','OUT OF SCOPE'],['Gun Shop','OUT OF SCOPE'],['Paintball Field','OUT OF SCOPE'],['Supermarket','OUT OF SCOPE'],['Pool','OUT OF SCOPE'],['Antique Shop','OUT OF SCOPE'],['General Entertainment','OUT OF SCOPE'],['Art Museum','OUT OF SCOPE'],['Scenic Lookout','OUT OF SCOPE'],['Boat or Ferry','OUT OF SCOPE'],['Optical Shop','OUT OF SCOPE'],['Stadium','OUT OF SCOPE'],['Martial Arts Dojo','OUT OF SCOPE'],['Skating Rink','OUT OF SCOPE'],['Airport','OUT OF SCOPE'],['Jewelry Store','OUT OF SCOPE'],['Auditorium','OUT OF SCOPE'],['Racetrack','OUT OF SCOPE'],['Beach','OUT OF SCOPE'],['Other Repair Shop','OUT OF SCOPE'],['River','OUT OF SCOPE'],['Convention Center','OUT OF SCOPE'],['Vineyard','OUT OF SCOPE'],['Stables','OUT OF SCOPE'],['Music Store','OUT OF SCOPE'],['Rental Car Location','OUT OF SCOPE'],['Bus Station','OUT OF SCOPE'],['Bowling Alley','OUT OF SCOPE'],['Laundry Service','OUT OF SCOPE'],['Arcade','OUT OF SCOPE'],['Other Great Outdoors','OUT OF SCOPE'],['Movie Theater','OUT OF SCOPE'],['Church','OUT OF SCOPE'],['Tennis Court','OUT OF SCOPE'],['Board Shop','OUT OF SCOPE'],['Gymnastics Gym','OUT OF SCOPE'],['Nail Salon','OUT OF SCOPE'],['Auto Workshop','OUT OF SCOPE'],['Metro Station','OUT OF SCOPE'],['Dog Run','OUT OF SCOPE'],['Locksmith','OUT OF SCOPE'],['Event Space','OUT OF SCOPE'],['Theme Park Ride / Attraction','OUT OF SCOPE'],['Fishing Store','OUT OF SCOPE'],['Soccer Stadium','OUT OF SCOPE'],['Flea Market','OUT OF SCOPE'],['Monument / Landmark','OUT OF SCOPE'],['Hookah Bar','OUT OF SCOPE'],['State / Provincial Park','OUT OF SCOPE'],['Accessories Store','OUT OF SCOPE'],['Neighborhood','OUT OF SCOPE'],['Recreation Center','OUT OF SCOPE'],['Real Estate Office','OUT OF SCOPE'],['Motel','OUT OF SCOPE'],['Record Shop','OUT OF SCOPE'],['College Gym','OUT OF SCOPE'],['IT Services','OUT OF SCOPE'],['Auto Dealership','OUT OF SCOPE'],['Sculpture Garden','OUT OF SCOPE'],['Big Box Store','OUT OF SCOPE'],['Golf Driving Range','OUT OF SCOPE'],['Canal','OUT OF SCOPE'],['Resort','OUT OF SCOPE'],['Baseball Stadium','OUT OF SCOPE']]
df_reviewedVenues=pd.DataFrame(scopelist, columns =['catname','bucket'])
df_reviewedVenues.head()

,catname,bucket
0,American Restaurant,Restaurant
1,Arepa Restaurant,Restaurant
2,Argentinian Restaurant,Restaurant
3,Asian Restaurant,Restaurant
4,Bagel Shop,Food Shop


### Join the Scope List and then remove the items that are out of scope.

In [13]:
# join the venues and the scope list
df_bcn_venues=pd.merge(df_bcn_venues,
    df_reviewedVenues,
    left_on='catname',
    right_on='catname',
    how='left')
df_bcn_venues.head(3)

,neighbourhood,name,lng,lat,catname,bucket
0,Abington,10th District Brewery,-70.945963,42.105673,Brewery,Barcentric
1,Abington,Spencer's Pizza,-70.939848,42.106542,Pizza Place,Grab and Go
2,Abington,Dairy Queen,-70.931984,42.109693,Ice Cream Shop,Food Shop


In [14]:
# Drop the out of scope entries
df_bcn_venues=df_bcn_venues[df_bcn_venues.bucket != 'OUT OF SCOPE']
df_bcn_venues=df_bcn_venues[['bucket','catname','name','neighbourhood','lat','lng']]
df_bcn_venues.head(3)

,bucket,catname,name,neighbourhood,lat,lng
0,Barcentric,Brewery,10th District Brewery,Abington,42.105673,-70.945963
1,Grab and Go,Pizza Place,Spencer's Pizza,Abington,42.106542,-70.939848
2,Food Shop,Ice Cream Shop,Dairy Queen,Abington,42.109693,-70.931984


### Limit the review to only the neighbourhoods with the highest count of in scope venues.

In [47]:
# Find the neighbourhoods with the most locations.  While crowded, these are likely also the most lively and viable for a new location of an under-represented type.
print('Venues per Neighbourhood:')
df_bcn_venues.groupby('neighbourhood')['bucket'].count().sort_values(ascending=False).head(10)

Venues per Neighbourhood:


neighbourhood
Quincy           72
Winchester       71
Beverly          70
Stoneham         70
North Andover    70
Revere           69
Somerville       68
Dracut           68
Melrose          68
Lowell           67
Name: bucket, dtype: int64

In [16]:
# Create a list of just the top 10 neighbourhoods to filter the venue list by.
top10_neighbourhoods_list=df_bcn_venues.groupby('neighbourhood')['bucket'].count().sort_values(ascending=False).head(10).index.tolist()
top10_neighbourhoods_list

['Quincy',
 'Winchester',
 'Beverly',
 'Stoneham',
 'North Andover',
 'Revere',
 'Somerville',
 'Dracut',
 'Melrose',
 'Lowell']

In [17]:
# Drop any venues which are not within the top 10 neighbourhoods.
df_top10_bcn_venues = df_bcn_venues[df_bcn_venues['neighbourhood'].isin(top10_neighbourhoods_list)]
df_top10_bcn_venues.reset_index(inplace=True)
df_top10_bcn_venues.head(2)

,index,bucket,catname,name,neighbourhood,lat,lng
0,950,Grab and Go,Pizza Place,Mike's Pizzeria Subs & Salad,Beverly,42.557763,-70.878962
1,951,Barcentric,Pub,The Indo Pub,Beverly,42.552174,-70.878930


### Determine the most common venue bucket across the remaining data.

In [18]:
# Create a new dataframe to allow for comparison of the rolled up bucket data.
bnc_onehot=pd.get_dummies(df_bcn_venues[['bucket']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
bnc_onehot['neighbourhood'] = df_top10_bcn_venues['neighbourhood']
bnc_onehot['lat'] = df_top10_bcn_venues['lat']
bnc_onehot['lng'] = df_top10_bcn_venues['lng']


# move neighbourhood column to the first column
fixed_columns = [bnc_onehot.columns[-1]] + list(bnc_onehot.columns[:-1])
bnc_onehot = bnc_onehot[fixed_columns]

#Create a grouped version for knn use later.
bnc_grouped = bnc_onehot.groupby('neighbourhood').mean().reset_index()

In [19]:
# Calculate the average bucket distribution so it can be added to the grouped dataframe which will be created shortly.
print('Venue distribution')
df_bcn_venues.groupby('bucket')['name'].count().sort_values(ascending=False)

Venue distribution


bucket
Restaurant     3013
Food Shop      1815
Grab and Go    1058
Barcentric      495
Other            36
Name: name, dtype: int64

In [20]:
#Adjust columns for legability, shifting lat and lng over and, since it's being modified, sorting the remaining columns by distribution.  (Doing it manually to save a step.)
bnc_grouped=bnc_grouped[['neighbourhood','lat','lng','Restaurant','Food Shop','Grab and Go','Barcentric','Other']]
bnc_grouped.head(2)

,neighbourhood,lat,lng,Restaurant,Food Shop,Grab and Go,Barcentric,Other
0,Beverly,42.544182,-70.888634,0.375000,0.250000,0.229167,0.12500,0.0
1,Dracut,42.657912,-71.313371,0.365854,0.439024,0.097561,0.04878,0.0


In [21]:
# ½ DEV CODE
bnc_grouped_bkp=pd.DataFrame(bnc_grouped)
bnc_grouped=pd.DataFrame(bnc_grouped_bkp)
bnc_grouped

,neighbourhood,lat,lng,Restaurant,Food Shop,Grab and Go,Barcentric,Other
0,Beverly,42.544182,-70.888634,0.375000,0.250000,0.229167,0.125000,0.000000
1,Dracut,42.657912,-71.313371,0.365854,0.439024,0.097561,0.048780,0.000000
2,Lowell,42.637230,-71.320284,0.525000,0.225000,0.125000,0.125000,0.000000
3,Melrose,42.453981,-71.070615,0.400000,0.325000,0.150000,0.050000,0.000000
4,North Andover,42.684433,-71.136782,0.510638,0.297872,0.127660,0.042553,0.000000
5,Quincy,42.252126,-71.007863,0.468085,0.319149,0.042553,0.106383,0.000000
6,Revere,42.398506,-71.022220,0.479167,0.270833,0.083333,0.104167,0.000000
7,Somerville,42.383845,-71.106528,0.348837,0.255814,0.255814,0.069767,0.023256
8,Stoneham,42.485558,-71.098157,0.526316,0.078947,0.263158,0.078947,0.000000
9,Winchester,42.446059,-71.137441,0.513514,0.216216,0.189189,0.081081,0.000000


In [22]:
# Create a dataframe with the average venue distribution percentages
df_venue_dsitribution=pd.DataFrame((df_bcn_venues.groupby('bucket')['name'].count()/df_bcn_venues['name'].count()).sort_values(ascending=False))
df_venue_dsitribution.rename({'name':'Target Pct'}, axis=1, inplace=True)
df_venue_dsitribution.reset_index(inplace=True)
df_venue_dsitribution.head()

,bucket,Target Pct
0,Restaurant,0.440948
1,Food Shop,0.265623
2,Grab and Go,0.154837
3,Barcentric,0.072443
4,Other,0.005269


In [23]:
for thisBucket in df_venue_dsitribution['bucket']:
    bnc_grouped[("Target % "+ thisBucket)]=max(df_venue_dsitribution['Target Pct'].loc[df_venue_dsitribution['bucket']==thisBucket])
    bnc_grouped[(thisBucket + " Viability Score")]=(max(df_venue_dsitribution['Target Pct'].loc[df_venue_dsitribution['bucket']==thisBucket])-bnc_grouped[thisBucket])
    bnc_grouped.loc[bnc_grouped[(thisBucket + " Viability Score")]<0,(thisBucket + " Viability Score")] = 0
bnc_grouped

,neighbourhood,lat,lng,Restaurant,Food Shop,Grab and Go,Barcentric,Other,Target % Restaurant,Restaurant Viability Score,Target % Food Shop,Food Shop Viability Score,Target % Grab and Go,Grab and Go Viability Score,Target % Barcentric,Barcentric Viability Score,Target % Other,Other Viability Score
0,Beverly,42.544182,-70.888634,0.375000,0.250000,0.229167,0.125000,0.000000,0.440948,0.065948,0.265623,0.015623,0.154837,0.000000,0.072443,0.000000,0.005269,0.005269
1,Dracut,42.657912,-71.313371,0.365854,0.439024,0.097561,0.048780,0.000000,0.440948,0.075095,0.265623,0.000000,0.154837,0.057276,0.072443,0.023662,0.005269,0.005269
2,Lowell,42.637230,-71.320284,0.525000,0.225000,0.125000,0.125000,0.000000,0.440948,0.000000,0.265623,0.040623,0.154837,0.029837,0.072443,0.000000,0.005269,0.005269
3,Melrose,42.453981,-71.070615,0.400000,0.325000,0.150000,0.050000,0.000000,0.440948,0.040948,0.265623,0.000000,0.154837,0.004837,0.072443,0.022443,0.005269,0.005269
4,North Andover,42.684433,-71.136782,0.510638,0.297872,0.127660,0.042553,0.000000,0.440948,0.000000,0.265623,0.000000,0.154837,0.027177,0.072443,0.029889,0.005269,0.005269
5,Quincy,42.252126,-71.007863,0.468085,0.319149,0.042553,0.106383,0.000000,0.440948,0.000000,0.265623,0.000000,0.154837,0.112284,0.072443,0.000000,0.005269,0.005269
6,Revere,42.398506,-71.022220,0.479167,0.270833,0.083333,0.104167,0.000000,0.440948,0.000000,0.265623,0.000000,0.154837,0.071503,0.072443,0.000000,0.005269,0.005269
7,Somerville,42.383845,-71.106528,0.348837,0.255814,0.255814,0.069767,0.023256,0.440948,0.092111,0.265623,0.009809,0.154837,0.000000,0.072443,0.002675,0.005269,0.000000
8,Stoneham,42.485558,-71.098157,0.526316,0.078947,0.263158,0.078947,0.000000,0.440948,0.000000,0.265623,0.186675,0.154837,0.000000,0.072443,0.000000,0.005269,0.005269
9,Winchester,42.446059,-71.137441,0.513514,0.216216,0.189189,0.081081,0.000000,0.440948,0.000000,0.265623,0.049406,0.154837,0.000000,0.072443,0.000000,0.005269,0.005269


### Highlight the under represented venue types and present the results.

In [24]:
bnc_summary=pd.DataFrame(bnc_grouped[['neighbourhood','lat','lng','Restaurant Viability Score','Food Shop Viability Score','Grab and Go Viability Score','Barcentric Viability Score','Other Viability Score']])
bnc_summary

,neighbourhood,lat,lng,Restaurant Viability Score,Food Shop Viability Score,Grab and Go Viability Score,Barcentric Viability Score,Other Viability Score
0,Beverly,42.544182,-70.888634,0.065948,0.015623,0.000000,0.000000,0.005269
1,Dracut,42.657912,-71.313371,0.075095,0.000000,0.057276,0.023662,0.005269
2,Lowell,42.637230,-71.320284,0.000000,0.040623,0.029837,0.000000,0.005269
3,Melrose,42.453981,-71.070615,0.040948,0.000000,0.004837,0.022443,0.005269
4,North Andover,42.684433,-71.136782,0.000000,0.000000,0.027177,0.029889,0.005269
5,Quincy,42.252126,-71.007863,0.000000,0.000000,0.112284,0.000000,0.005269
6,Revere,42.398506,-71.022220,0.000000,0.000000,0.071503,0.000000,0.005269
7,Somerville,42.383845,-71.106528,0.092111,0.009809,0.000000,0.002675,0.000000
8,Stoneham,42.485558,-71.098157,0.000000,0.186675,0.000000,0.000000,0.005269
9,Winchester,42.446059,-71.137441,0.000000,0.049406,0.000000,0.000000,0.005269


In [25]:
# Calculate the average bucket distribution so it can be added to the grouped dataframe which will be created shortly.
#print('Venue distribution')
#bnc_summary.sort_values(by=['Restaurant Viability Score'], ascending=False)

In [28]:
# Melt the summary and drop all zero viability lines.
bnc_viability_summary=pd.DataFrame()
bnc_viability_summary=bnc_summary.melt(id_vars=['neighbourhood','lat','lng'],var_name='Viability Type',value_name='Viability Percent')
bnc_viability_summary=bnc_viability_summary[bnc_viability_summary['Viability Percent'] != 0]
bnc_viability_summary.sort_values(by=['Viability Percent','neighbourhood'], ascending=False, inplace=True)
bnc_viability_summary.reset_index(inplace=True, drop=True)
bnc_viability_summary

,neighbourhood,lat,lng,Viability Type,Viability Percent
0,Stoneham,42.485558,-71.098157,Food Shop Viability Score,0.186675
1,Quincy,42.252126,-71.007863,Grab and Go Viability Score,0.112284
2,Somerville,42.383845,-71.106528,Restaurant Viability Score,0.092111
3,Dracut,42.657912,-71.313371,Restaurant Viability Score,0.075095
4,Revere,42.398506,-71.022220,Grab and Go Viability Score,0.071503
5,Beverly,42.544182,-70.888634,Restaurant Viability Score,0.065948
6,Dracut,42.657912,-71.313371,Grab and Go Viability Score,0.057276
7,Winchester,42.446059,-71.137441,Food Shop Viability Score,0.049406
8,Melrose,42.453981,-71.070615,Restaurant Viability Score,0.040948
9,Lowell,42.637230,-71.320284,Food Shop Viability Score,0.040623


### Investigate the top 3 options.

In [29]:
bnc_viability_summary.head(3)

,neighbourhood,lat,lng,Viability Type,Viability Percent
0,Stoneham,42.485558,-71.098157,Food Shop Viability Score,0.186675
1,Quincy,42.252126,-71.007863,Grab and Go Viability Score,0.112284
2,Somerville,42.383845,-71.106528,Restaurant Viability Score,0.092111


In [30]:
# Find the most common Food Shops across the original target area.
print('Most Popular Food Shop Options')
df_onlyFoodShops = df_bcn_venues[df_bcn_venues['bucket']=='Food Shop']
df_top5_FoodShops=pd.DataFrame(df_onlyFoodShops.groupby('name')['bucket'].count().sort_values(ascending=False).head())
df_top5_FoodShops.drop(columns='bucket',inplace=True)
df_top5_FoodShops.reset_index(inplace=True)
df_top5_FoodShops

Most Popular Food Shop Options


,name
0,Dunkin'
1,Starbucks
2,Panera Bread
3,Dairy Queen
4,Honey Dew Donuts


In [31]:
# Find the most common Grab and Gos across the original target area.
print('Most Popular Grab and Go Options')
df_onlyGandG = df_bcn_venues[df_bcn_venues['bucket']=='Grab and Go']
df_top5_GandG=pd.DataFrame(df_onlyGandG.groupby('name')['bucket'].count().sort_values(ascending=False).head(6))
df_top5_GandG.drop(columns='bucket',inplace=True)
df_top5_GandG.reset_index(inplace=True)
df_top5_GandG

Most Popular Grab and Go Options


,name
0,SUBWAY
1,Five Guys
2,Domino's Pizza
3,Subway
4,Papa Gino's
5,D'Angelo Grilled Sandwiches


In [32]:
# Find the most common Restaurants across the original target area.
print('Most Popular Restaurant Options')
df_onlyRestaurant = df_bcn_venues[df_bcn_venues['bucket']=='Restaurant']
df_top5_Restaurant=pd.DataFrame(df_onlyRestaurant.groupby('name')['bucket'].count().sort_values(ascending=False).head(6))
df_top5_Restaurant.drop(columns='bucket',inplace=True)
df_top5_Restaurant.reset_index(inplace=True)
df_top5_Restaurant

Most Popular Restaurant Options


,name
0,Chipotle Mexican Grill
1,Ninety Nine Restaurant
2,McDonald's
3,Bertucci's
4,Wendy’s
5,LongHorn Steakhouse


# Review Option 1

In [33]:
bnc_viability_summary.loc[0,:]

neighbourhood                         Stoneham
lat                                    42.4856
lng                                   -71.0982
Viability Type       Food Shop Viability Score
Viability Percent                     0.186675
Name: 0, dtype: object

In [34]:
df_Option1_existing=df_bcn_venues[(df_bcn_venues['bucket']=='Food Shop') & (df_bcn_venues['neighbourhood']=='Stoneham')]
df_Option1_existing

,bucket,catname,name,neighbourhood,lat,lng
9310,Food Shop,Coffee Shop,Kushala Sip Coffee House,Stoneham,42.480803,-71.100373
9313,Food Shop,Candy Store,Winfrey's Fudge & Chocolates,Stoneham,42.499152,-71.101230
9326,Food Shop,Deli / Bodega,Billy's Famous Roast Beef,Stoneham,42.473554,-71.061409
9331,Food Shop,Ice Cream Shop,Cravings,Stoneham,42.503490,-71.070287
9332,Food Shop,Bakery,Gingerbread Construction Co.,Stoneham,42.467550,-71.125543
9350,Food Shop,Café,Creations,Stoneham,42.503337,-71.070276
9358,Food Shop,Deli / Bodega,D'Agostinos,Stoneham,42.451643,-71.138232
9365,Food Shop,Bakery,D'Amici's Bakery,Stoneham,42.454408,-71.065848
9368,Food Shop,Ice Cream Shop,Nanabette's Ice Cream,Stoneham,42.472409,-71.150123
9372,Food Shop,Bakery,swissbäkers,Stoneham,42.521795,-71.107295


In [35]:
#Display the data on a map of the area.
df_thisOption=df_Option1_existing

thisOptionlat=df_thisOption['lat'].mean()
thisOptionlng=df_thisOption['lng'].mean()

thisOptionMap = folium.Map(location=[thisOptionlat, thisOptionlng], zoom_start=14)


# set color scheme for the clusters
rainbow = ['#ffffff','#ff0000']

# add markers to the map
markers_colors = []
thiscolor=1
for lat, lon, poi, cluster in zip(df_thisOption['lat'],df_thisOption['lng'], df_thisOption['name'], df_thisOption['bucket']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[thiscolor],
        fill=True,
        fill_color=rainbow[thiscolor],
        fill_opacity=0.7).add_to(thisOptionMap)
       
thisOptionMap

In [36]:
df_Option1=pd.merge(df_top5_FoodShops,
                    df_Option1_existing,
                    left_on='name',
                    right_on='name',
                    how='left',
                    indicator=True)
df_Option1


,name,bucket,catname,neighbourhood,lat,lng,_merge
0,Dunkin',Food Shop,Donut Shop,Stoneham,42.476742,-71.090646,both
1,Dunkin',Food Shop,Donut Shop,Stoneham,42.508829,-71.124342,both
2,Starbucks,NaN,NaN,NaN,NaN,NaN,left_only
3,Panera Bread,NaN,NaN,NaN,NaN,NaN,left_only
4,Dairy Queen,NaN,NaN,NaN,NaN,NaN,left_only
5,Honey Dew Donuts,NaN,NaN,NaN,NaN,NaN,left_only


### According to the assembled foursquare data, there is a clear gap in Stoneham for a Starbucks, Panera Bread or a Dairy Queen.  The obvious choice of a Starbucks was thrown out due to suspected api limits causing it's absense.  Also, a Honey Dew Donuts is not recommended due to the competition from the two Dunkin' Donuts already present.  For those reasons, a Panera Bread is suggested.


# Review Option 2

In [37]:
bnc_viability_summary.loc[1,:]

neighbourhood                             Quincy
lat                                      42.2521
lng                                     -71.0079
Viability Type       Grab and Go Viability Score
Viability Percent                       0.112284
Name: 1, dtype: object

In [38]:
df_Option2_existing=df_bcn_venues[(df_bcn_venues['bucket']=='Grab and Go') & (df_bcn_venues['neighbourhood']=='Quincy')]
df_Option2_existing

,bucket,catname,name,neighbourhood,lat,lng
8001,Grab and Go,Burger Joint,Five Guys,Quincy,42.251781,-71.003171
8025,Grab and Go,Sandwich Place,SUBWAY,Quincy,42.250684,-71.002901
8053,Grab and Go,Sandwich Place,Jersey Mike's Subs,Quincy,42.247630,-70.982030
8064,Grab and Go,Pizza Place,School House Pizza,Quincy,42.245343,-71.000385
8068,Grab and Go,Pizza Place,I Love Italian Pizza,Quincy,42.281352,-71.029071
8073,Grab and Go,Pizza Place,Balducci's House of Pizza,Quincy,42.272599,-71.026510
8083,Grab and Go,Sandwich Place,D'Angelo Grilled Sandwiches,Quincy,42.254594,-70.995351
8085,Grab and Go,Pizza Place,Pizza Connection,Quincy,42.271139,-71.030244


In [39]:
#Display the data on a map of the area.
df_thisOption=df_Option2_existing

thisOptionlat=df_thisOption['lat'].mean()
thisOptionlng=df_thisOption['lng'].mean()

thisOptionMap = folium.Map(location=[thisOptionlat, thisOptionlng], zoom_start=14)


# set color scheme for the clusters
rainbow = ['#ffffff','#ff00ff']

# add markers to the map
markers_colors = []
thiscolor=1
for lat, lon, poi, cluster in zip(df_thisOption['lat'],df_thisOption['lng'], df_thisOption['name'], df_thisOption['bucket']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[thiscolor],
        fill=True,
        fill_color=rainbow[thiscolor],
        fill_opacity=0.7).add_to(thisOptionMap)
       
thisOptionMap

In [40]:
df_Option2=pd.merge(df_top5_GandG,
                    df_Option2_existing,
                    left_on='name',
                    right_on='name',
                    how='left',
                    indicator=True)
df_Option2

,name,bucket,catname,neighbourhood,lat,lng,_merge
0,SUBWAY,Grab and Go,Sandwich Place,Quincy,42.250684,-71.002901,both
1,Five Guys,Grab and Go,Burger Joint,Quincy,42.251781,-71.003171,both
2,Domino's Pizza,NaN,NaN,NaN,NaN,NaN,left_only
3,Subway,NaN,NaN,NaN,NaN,NaN,left_only
4,Papa Gino's,NaN,NaN,NaN,NaN,NaN,left_only
5,D'Angelo Grilled Sandwiches,Grab and Go,Sandwich Place,Quincy,42.254594,-70.995351,both


### Despite the high viability indicator, Quincy seems overstocked in both sandwhich shops and pizza places.  However, with only Five Guys visable, there may be room for an aditional fast food place.  Further research would need to occur.

# Review Option 3 (Grab and Go in Dedham)

In [41]:
bnc_viability_summary.loc[2,:]

neighbourhood                        Somerville
lat                                     42.3838
lng                                    -71.1065
Viability Type       Restaurant Viability Score
Viability Percent                     0.0921111
Name: 2, dtype: object

In [42]:
df_Option3_existing=df_bcn_venues[(df_bcn_venues['bucket']=='Restaurant') & (df_bcn_venues['neighbourhood']=='Somerville')]
df_Option3_existing

,bucket,catname,name,neighbourhood,lat,lng
9049,Restaurant,Meze Restaurant,Sarma,Somerville,42.388163,-71.095664
9050,Restaurant,New American Restaurant,Highland Kitchen,Somerville,42.388270,-71.103240
9061,Restaurant,Brazilian Restaurant,Pastelaria Broadway,Somerville,42.389116,-71.087296
9066,Restaurant,Dumpling Restaurant,Tasty Momo,Somerville,42.396284,-71.103491
9068,Restaurant,Tapas Restaurant,Dali,Somerville,42.378327,-71.106661
9075,Restaurant,Japanese Restaurant,Yume Ga Arukara,Somerville,42.387210,-71.118467
9076,Restaurant,Mexican Restaurant,Taco Loco,Somerville,42.386502,-71.079759
9084,Restaurant,Vegetarian / Vegan Restaurant,Taco Party,Somerville,42.400052,-71.112461
9087,Restaurant,Mediterranean Restaurant,Oleana,Somerville,42.370658,-71.097168
9099,Restaurant,Greek Restaurant,Opa Greek Yeeros,Somerville,42.395093,-71.119122


In [43]:
#Display the data on a map of the area.
df_thisOption=df_Option3_existing

thisOptionlat=df_thisOption['lat'].mean()
thisOptionlng=df_thisOption['lng'].mean()

thisOptionMap = folium.Map(location=[thisOptionlat, thisOptionlng], zoom_start=14)


# set color scheme for the clusters
rainbow = ['#ffffff','#0000ff']

# add markers to the map
markers_colors = []
thiscolor=1
for lat, lon, poi, cluster in zip(df_thisOption['lat'],df_thisOption['lng'], df_thisOption['name'], df_thisOption['bucket']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[thiscolor],
        fill=True,
        fill_color=rainbow[thiscolor],
        fill_opacity=0.7).add_to(thisOptionMap)
       
thisOptionMap

In [45]:
df_Option3=pd.merge(df_top5_Restaurant,
                    df_Option3_existing,
                    left_on='name',
                    right_on='name',
                    how='left',
                    indicator=True)
df_Option3

,name,bucket,catname,neighbourhood,lat,lng,_merge
0,Chipotle Mexican Grill,NaN,NaN,NaN,NaN,NaN,left_only
1,Ninety Nine Restaurant,NaN,NaN,NaN,NaN,NaN,left_only
2,McDonald's,NaN,NaN,NaN,NaN,NaN,left_only
3,Bertucci's,NaN,NaN,NaN,NaN,NaN,left_only
4,Wendy’s,NaN,NaN,NaN,NaN,NaN,left_only
5,LongHorn Steakhouse,NaN,NaN,NaN,NaN,NaN,left_only


### The Somerville looks extremely promising.  However, the absense of all major resturaunts is unsettling.  Manually verifying these results uncovered that this is a result of limits on the free version of the foursquare api, and not an actual derth of venues.  It was found that the nearest Wendy's is outside of the Somerville target area and thus would be a very good option here. 

# Conclusion

# Map of Option Locations

In [46]:
#Display the data on a map of the area.
pieces={'1':df_Option1_existing, '2':df_Option2_existing, '3':df_Option3_existing}
df_thisOption = pd.concat(pieces, keys=['1', '2', '3'],
                   levels=[['1', '2', '3', '4']],
                   names=['group_key'])
df_thisOption.reset_index(inplace=True)

thisOptionlat=df_thisOption['lat'].mean()
thisOptionlng=df_thisOption['lng'].mean()

thisOptionMap = folium.Map(location=[thisOptionlat, thisOptionlng], zoom_start=11)

# set color scheme for the clusters
rainbow = ['#ffffff','#ff0000','#ff00ff','#0000ff']

# add markers to the map
markers_colors = []
thiscolor=1
for lat, lon, poi, cluster in zip(df_thisOption['lat'],df_thisOption['lng'], df_thisOption['name'], df_thisOption['group_key'].astype(int)):
    label = folium.Popup(str(poi) + ' Option ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(thisOptionMap)
       
thisOptionMap

Although this effort has been hobbled by the data limits of the free version of the foursquare api, the following suggestions and are presented in decending .

1. Open a Panera Bread near Stoneham. (Red - Option 1)
2. Open a Wendy's near Somerville. (Blue - Option 3)
3. Explore franchise options to open a burger joint near Quincy (Pink - Option 2)

# Appendix

The map below shows the intended coverage of the queries.  However, due to restrictions on the free version of the foursquare api, each area was limited to 100 items max.  This created vital gaps in knowledge. (e.g. The McDonalds near Somerville was likely one of the 137 venues left out of that data.).  If this were a real life effort, it may make sense to pay for the full api to gain not only the full search results, but also insight into visitor numbers and other helpful metrics.

In [51]:
df_query_centers=df_bcn_venues.groupby('neighbourhood')['lat','lng'].mean()
df_query_centers

,lat,lng
neighbourhood,,
Abington,42.101985,-70.952310
Acton,42.484735,-71.424253
Amesbury,42.848456,-70.917474
Andover,42.669800,-71.138492
Arlington,42.409697,-71.141727
Ashby,42.695991,-71.803983
Ashland,42.272373,-71.442480
Avon,42.130685,-71.044646
Ayer,42.556782,-71.585425


In [59]:
#Display the data on a map of the area.
df_query_centers=df_bcn_venues.groupby('neighbourhood')['lat','lng'].mean()

thisOptionlat=df_query_centers['lat'].mean()
thisOptionlng=df_query_centers['lng'].mean()

thisOptionMap = folium.Map(location=[thisOptionlat, thisOptionlng], zoom_start=11)


# set color scheme for the clusters
rainbow = ['#ffffff','#ff0000']

# add markers to the map
markers_colors = []
thiscolor=1
for lat, lon, poi, cluster in zip(df_thisOption['lat'],df_thisOption['lng'], df_thisOption['name'], df_thisOption['bucket']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.Circle(
        [lat, lon],
        radius=5000,
        popup=label,
        color=rainbow[thiscolor],
        fill=True,
        fill_color=rainbow[thiscolor],
        fill_opacity=0.1).add_to(thisOptionMap)
       
thisOptionMap